In [13]:
# Make a copick project

config_blob = """{
    "name": "czii_cryoet_mlchallenge_2024",
    "description": "2024 CZII CryoET ML Challenge training data.",
    "version": "1.0.0",

    "pickable_objects": [
        {
            "name": "apo-ferritin",
            "is_particle": true,
            "pdb_id": "4V1W",
            "label": 1,
            "color": [  0, 117, 220, 128],
            "radius": 60,
            "map_threshold": 0.0418
        },
        {
            "name": "beta-amylase",
            "is_particle": true,
            "pdb_id": "1FA2",
            "label": 2,
            "color": [153,  63,   0, 128],
            "radius": 65,
            "map_threshold": 0.035
        },
        {
            "name": "beta-galactosidase",
            "is_particle": true,
            "pdb_id": "6X1Q",
            "label": 3,
            "color": [ 76,   0,  92, 128],
            "radius": 90,
            "map_threshold": 0.0578
        },
        {
            "name": "ribosome",
            "is_particle": true,
            "pdb_id": "6EK0",
            "label": 4,
            "color": [  0,  92,  49, 128],
            "radius": 150,
            "map_threshold": 0.0374
        },
        {
            "name": "thyroglobulin",
            "is_particle": true,
            "pdb_id": "6SCJ",
            "label": 5,
            "color": [ 43, 206,  72, 128],
            "radius": 130,
            "map_threshold": 0.0278
        },
        {
            "name": "virus-like-particle",
            "is_particle": true,
            "label": 6,
            "color": [255, 204, 153, 128],
            "radius": 135,
            "map_threshold": 0.201
        },
        {
            "name": "membrane",
            "is_particle": false,
            "label": 8,
            "color": [100, 100, 100, 128]
        },
        {
            "name": "background",
            "is_particle": false,
            "label": 9,
            "color": [10, 150, 200, 128]
        }
    ],

    "overlay_root": "overlay",

    "overlay_fs_args": {
        "auto_mkdir": true
    },

    "static_root": "/bigwork/nhwpajjy/competition/cryoET-data/czii-cryo-et-object-identification/train/static"
}"""

copick_config_path = "train_copick.config"
output_overlay = "overlay"

with open(copick_config_path, "w") as f:
    f.write(config_blob)

In [2]:
# Setup new overlay directory
import os
import shutil

# Define source and destination directories
source_dir = '/bigwork/nhwpajjy/competition/cryoET-data/czii-cryo-et-object-identification/train/overlay'
destination_dir = 'overlay'

# Walk through the source directory
for root, dirs, files in os.walk(source_dir):
    # Create corresponding subdirectories in the destination
    relative_path = os.path.relpath(root, source_dir)
    target_dir = os.path.join(destination_dir, relative_path)
    os.makedirs(target_dir, exist_ok=True)
    
    # Copy and rename each file
    for file in files:
        if file.startswith("curation_0_"):
            new_filename = file
        else:
            new_filename = f"curation_0_{file}"
            
        
        # Define full paths for the source and destination files
        source_file = os.path.join(root, file)
        destination_file = os.path.join(target_dir, new_filename)
        
        # Copy the file with the new name
        shutil.copy2(source_file, destination_file)
        print(f"Copied {source_file} to {destination_file}")


Copied /bigwork/nhwpajjy/competition/cryoET-data/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_6_4/Picks/ribosome.json to overlay/ExperimentRuns/TS_6_4/Picks/curation_0_ribosome.json
Copied /bigwork/nhwpajjy/competition/cryoET-data/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_6_4/Picks/virus-like-particle.json to overlay/ExperimentRuns/TS_6_4/Picks/curation_0_virus-like-particle.json
Copied /bigwork/nhwpajjy/competition/cryoET-data/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_6_4/Picks/apo-ferritin.json to overlay/ExperimentRuns/TS_6_4/Picks/curation_0_apo-ferritin.json
Copied /bigwork/nhwpajjy/competition/cryoET-data/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_6_4/Picks/beta-amylase.json to overlay/ExperimentRuns/TS_6_4/Picks/curation_0_beta-amylase.json
Copied /bigwork/nhwpajjy/competition/cryoET-data/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_6_4/Picks/thyroglobulin.json t

In [3]:
from deepfindET.entry_points import step1
from deepfindET.utils import copick_tools
import matplotlib.pyplot as plt
import copick

config = 'copick.config'

# Query Tomogram
voxel_size = 10 
tomogram_algorithm = 'denoised'

# Output Name for the Segmentation Targets
out_name = 'remotetargets'
out_user_id = 'deepfindET'
out_session_id = '0'

# Read Copick Directory
copickRoot = copick.from_file(config)


train_targets = {}
targets = [(obj.name, None, None, (obj.radius / voxel_size)) for obj in copickRoot.pickable_objects if obj.is_particle]
validate_runs = "TS_86_3"
run_ids = [run.name for run in copickRoot.runs if run.name != validate_runs]


In [28]:
# Generate train target information
for t in targets:
    obj_name, user_id, session_id, radius = t
    info = {
        "label": copickRoot.get_object(obj_name).label,
        "user_id": user_id,
        "session_id": session_id,
        "radius": radius,
        "is_particle_target": True,
    }
    train_targets[obj_name] = info


# Define segmentation target (e.g., membrane)
seg_targets = [('membrane', None, None)]

# Generate segmentation target information
for s in seg_targets:
    obj_name, user_id, session_id = s
    info = {
        "label": copickRoot.get_object(obj_name).label,
        "user_id": user_id,
        "session_id": session_id,
        "radius": None,       
        "is_particle_target": False,                 
    }
    train_targets[obj_name] = info

step1.create_train_targets(
    config,              # The configuration file path specifying various settings and parameters for the project.
    train_targets,       # A dictionary containing the target information for each protein or object to be segmented.
    run_ids,             # The list of Run-IDs for which to generate targets. None means targets for the entire project.
    voxel_size,          # The voxel size to be used in the tomogram data.
    tomogram_algorithm,  # The reconstruction algorithm used for the tomograms, e.g., 'wbp' (weighted back projection).
    out_name,            # The output name for the generated segmentation targets.
    out_user_id,         # The user ID under which the output targets will be saved.
    out_session_id,      # The session ID associated with the output, typically used for tracking purposes.
)

Processing Run: TS_5_4
Annotating 140 objects ...

Processing Run: TS_69_2
Annotating 143 objects ...

Processing Run: TS_6_4
Annotating 193 objects ...

Processing Run: TS_6_6
Annotating 143 objects ...

Processing Run: TS_73_6
Annotating 217 objects ...

Processing Run: TS_86_3
Annotating 225 objects ...

Processing Run: TS_99_9
Annotating 208 objects ...



In [11]:
# Option 1: Query All RunIDs
# Retrieve all available Run-IDs from the CoPick project. This generates a list of Run-IDs by iterating over all runs in copickRoot.






# Option 2: Manually Specify Specific Run
# Define a specific Run-ID manually. This is useful for extracting volumes for a specific run.
runID = 'TS_6_6'

# Retrieve the specific run object from CoPick using the manually specified Run-ID.
copick_run = copickRoot.get_run(runID)

# Extract the segmentation target associated with the specified run.
# The function get_copick_segmentation retrieves the segmentation data (e.g., target volume) based on the run object,
# segmentation name, user ID, and session ID.
train_target = copick_tools.get_copick_segmentation(
    copick_run,                 # The run object obtained from CoPick for the specific Run-ID.
    segmentationName='remotetargets',  # The name of the segmentation target to retrieve.
    userID='deepfindET',        # The user ID under which the segmentation data is saved.
    sessionID='0'               # The session ID associated with the segmentation data.
)

# Retrieve the tomogram associated with the specified Run-ID from the CoPick project.
# The function get_copick_tomogram extracts the tomogram data, using the voxel size, algorithm, and Run-ID.
train_tomogram = copick_tools.get_copick_tomogram(
    copickRoot,                 # The root object for the CoPick project, containing all runs and associated data.
    voxelSize=voxel_size,       # The voxel size to be used for retrieving the tomogram.
    tomoAlgorithm='wbp',        # The reconstruction algorithm used for the tomogram, e.g., 'wbp' (weighted back projection).
    tomoID=runID                # The specific Run-ID for which the tomogram is being retrieved.
)

['TS_5_4', 'TS_69_2', 'TS_6_4', 'TS_6_6', 'TS_73_6', 'TS_99_9']

In [30]:
from deepfindET.entry_points import step2

# Specify the directory where the training results will be saved.
training_output_path = 'train_results'

# Set the model architecture to Residual U-Net ('res_unet'), 
# which combines U-Net with residual connections to improve training.
model_name = 'res_unet'

# Set to None to indicate that the model will betrained from scratch 
# without using pre-trained weights.
model_pre_weights = None

# Number of classes the model will predict. 
# Here, we are working with 8 different classes (6 proteins + membrane + background).
n_class = 8

# Input dimension size of the 3D volumes in voxels. Each input is a 72x72x72 voxel cube -- (72 Å)^3.
dim_in = 72  # [voxels]

2025-01-14 20:52:56.297854: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736884378.238037 1493925 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736884379.484853 1493925 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-14 20:53:12.835506: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [31]:
# Initiate the training of the DeepFindET 3D U-Net model.
step2.train_model(
    config,                 # Configuration file with various settings for the project.
    voxel_size,             # Voxel size used in the tomogram data.
    tomogram_algorithm,     # Reconstruction algorithm used for the tomograms (e.g., 'wbp').
    targets,                 # Target data for training the model.
    training_output_path,   # Path where the training outputs will be saved.
    model_name,             # Model architecture name ('res_unet').
    model_pre_weights,      # Pre-trained weights (None for training from scratch).
    n_class,                # Number of classes for segmentation (8 in this case).
    path_valid=None,        # Path to validation data (None means internal splitting may be used).
    dim_in=dim_in,          # Input dimension size in voxels.
    n_sub_epoch=10,         # Number of epochs to train on tomograms prior to swapping to a new set of tomograms.
    sample_size=3,          # Number of tomograms to extract per epoch.
    batch_size=10,          # Batch size used during training.
    epochs=20,              # Total number of training epochs.
    steps_per_epoch=2,    # Number of steps per epoch.
    n_valid=20,             # Number of validation samples.
    model_filters=[48, 64, 128],  # Filters in the convolutional layers at each level of the U-Net.
    model_dropout=0,        # Dropout rate (0 means no dropout applied).
    target_name="remotetargets",    # Name of the segmentation targets.
    target_user_id="deepfindET",  # User ID for the segmentation labels.
    target_session_id="0",    # Session ID associated with the labeling.
    valid_tomo_ids=None,    # List of tomogram IDs for validation.
    train_tomo_ids=None,     # List of tomogram IDs for training.
    class_weights=(('apo-ferritin', 62400), ('beta-amylase', 4130), ('beta-galactosidase', 3080), ('ribosome', 1800), ('thyroglobulin', 10100), ('virus-like-particle', 8400))
)
# 10439:     # class_weights=(('membrane',1),('adp-mitochondrial',3000),('alkaline-phosphate',3000),('nucleosome',3000),('ribosome',750),('vault',500),('virus-like-capsid',750))

I0000 00:00:1736884628.881190 1493925 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79196 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:c3:00.0, compute capability: 8.0



Training res_unet with Randomly Initialized Weights

Physical devices cannot be modified after being initialized
Loading Targets and Tomograms for the Following Runs: ['TS_6_4', 'TS_99_9', 'TS_73_6']


100%|██████████| 3/3 [00:06<00:00,  2.26s/it]


Loading Targets and Tomograms for the Following Runs: ['TS_5_4', 'TS_69_2']


100%|██████████| 2/2 [00:04<00:00,  2.39s/it]


...

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 150ms/step
Swapping datasets at epoch 20
Loading Targets and Tomograms for the Following Runs: ['TS_6_6', 'TS_99_9', 'TS_73_6']


100%|██████████| 3/3 [00:00<00:00,  4.96it/s]


Loading Targets and Tomograms for the Following Runs: ['TS_5_4', 'TS_69_2']


100%|██████████| 2/2 [00:00<00:00,  5.42it/s]


2/2 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step - accuracy: 0.8685 - loss: 2636.8215 - val_accuracy: 0.2313 - val_loss: 7.7016 - learning_rate: 1.0000e-04


In [32]:
from deepfindET.utils import core
import h5py, os

# Set the path to the training history file
history_path = os.path.join(training_output_path, 'net_train_history.h5')

# Convert the HDF5 file containing the training history into a dictionary format
# This allows easy access to the training metrics like loss, accuracy, etc., stored during training
history = core.convert_hdf5_to_dictionary(history_path)

# Plot the training history to visualize the learning process
# The plot_history function will generate curves for metrics such as training and validation loss, accuracy, etc.
core.plot_history(history, save_figure=False)
plt.show()

In [33]:
!ls /kaggle/working/train_results

ls: cannot access '/kaggle/working/train_results': No such file or directory


In [7]:


patch_size = 160            # Size of the input patch fed into the model for inference.
model_name = 'res_unet'     # The model architecture used for training.
filters = [48, 64, 128]      # Number of filters for U-Net (same parameter as used for training).
dropout = 0                 # Dropout rate applied during inference.

# Path to the pre-trained model weights for the chosen architecture.
model_weights = 'train_results/net_weights_FINAL.h5'

# Query for Tomogram
voxel_size = 10             # Resolution of the tomogram in voxel size.
tomogram_algorithm = 'denoised'  # Reconstruction algorithm used for generating the tomogram

# Output Segmentation Write Name
segmentation_name = 'predict'
session_id = '0'
user_id = 'deepfindET'



In [36]:
# Run the train DeepFindET 3D U-Net model on the copick directory.
from deepfindET.entry_points import step3
step3.inference_tomogram_segmentation(
    config,                                 # Copick Configuration File
    n_class,                                # Number of classes to predict.
    model_name,                             # The model architecture used for training.
    model_weights,                          # Path to the pre-trained model weights for the chosen architecture.
    patch_size,                             # Size of the input patch fed into the model for inference.
    user_id,                                # Identifier of the user or project running the segmentation.
    session_id,                             # Session identifier for tracking purposes.
    segmentation_name=segmentation_name,     # Identifier for the output segmentation file.
    voxel_size = voxel_size,                # Voxel Size of Tomogram to Run Inference On       
    model_filters = filters,                # Number of filters for U-Net
    model_dropout = dropout,                # Dropout rate
    tomogram_algorithm= tomogram_algorithm, # Reconstruction algorithm used for generating the tomogram
)


Training res_unet with train_results/net_weights_FINAL.h5 Weights


Segmentation Parameters:  {
    "input": {
        "predict_config": "copick.config",
        "voxel_size": 10,
        "tomogram_algorithm": "denoised"
    },
    "model_architecture": {
        "n_class": 8,
        "model_name": "res_unet",
        "path_weights": "train_results/net_weights_FINAL.h5",
        "patch_size": 160,
        "model_filters": [
            48,
            64,
            128
        ],
        "model_dropout": 0
    },
    "output": {
        "user_id": "deepfindET",
        "session_id": "0",
        "output_scoremap": false,
        "scoremap_name": "scoremap",
        "segmentation_name": "predict",
        "tomo_ids": null
    }
} 


Processing Run: TS_5_4 (0/3)
Data array is divided in 72 patches ...
Segmenting patch 1 / 72 ...
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
Segmenting patch 2 / 72 ...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Segmenting patch 3 / 72 ...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s

In [8]:
from deepfindET.entry_points import step4

copick_root = copick.from_file(config)

# Session ID for the Output Picks
picks_session_id = '0'

segmentation_session_id = '0'

segmentation_name = 'predict' 

min_protein_size = 0.4  

path_output = f"{copick_root.root_overlay}/ExperimentRuns"

In [9]:
[obj.radius for obj in copick_root.pickable_objects]

[60.0, 65.0, 90.0, 150.0, 130.0, 135.0]

In [10]:
# Derived from https://github.com/copick/DeepFindET/blob/main/deepfindET/entry_points/step4.py

import deepfindET.utils.copick_tools as tools
import deepfindET.utils.evaluate as evaluate
import scipy.ndimage as ndimage
from tqdm import tqdm
import numpy as np
# Currently Filtering Process always finds coordinate at (cx,cy,cz) - center coordinate
# This seems to always be at the first row, so we can remove it 
remove_index = 0

# Extract Protein Coordinates from the Segmentation Masks
def extract_coords(pickable_object, copick_run, segmentation_name):
    labelmap = tools.get_copick_segmentation(copick_run, segmentation_name, user_id, segmentation_session_id)[:]
    label = pickable_object.label
    protein_name = pickable_object.name
    label_objs, _ = ndimage.label(labelmap == label)

    # Filter Candidates based on Object Size
    # Get the sizes of all objects
    object_sizes = np.bincount(label_objs.flat)

    # Filter the objects based on size
    min_object_size = 4/3 * np.pi * ((pickable_object.radius/voxel_size)**2) * min_protein_size
    valid_objects = np.where(object_sizes > min_object_size)[0]                          

    # Estimate Coordiantes from CoM for LabelMaps
    deepFinderCoords = []
    for object_num in tqdm(valid_objects):
        com = ndimage.center_of_mass(label_objs == object_num)
        swapped_com = (com[2], com[1], com[0])
        deepFinderCoords.append(swapped_com)
    deepFinderCoords = np.array(deepFinderCoords)   

    # For some reason, consistently extracting center coordinate
    # Remove the row with the closest index
    deepFinderCoords = np.delete(deepFinderCoords, remove_index, axis=0)                    

    # Estimate Distance Threshold Based on 1/2 of Particle Diameter
    threshold = np.ceil(  pickable_object.radius / (voxel_size * 3) )

    try: 
        # Remove Double Counted Coordinates
        deepFinderCoords = evaluate.remove_repeated_picks(deepFinderCoords, threshold)

        # Append Euler Angles to Coordinates [ Expand Dimensions from Nx3 -> Nx6 ]
        deepFinderCoords = np.concatenate((deepFinderCoords, np.zeros(deepFinderCoords.shape)),axis=1)

        # Convert from Voxel to Physical Units
        deepFinderCoords *= voxel_size

    except Exception as e:
        print(f"Error processing label {label} in tomo {copick_run}: {e}")
        deepFinderCoords = np.array([]).reshape(0,6)

    # Save Picks in Copick Format / Directory 
    tools.write_copick_output(protein_name, copick_run.meta.name, deepFinderCoords, path_output, pickMethod=user_id, sessionID = picks_session_id)
    
    
for run in copick_root.runs:
    print(f"Run {run}")
    for pickable_object in copick_root.pickable_objects:
        print(pickable_object.name)
        if pickable_object.is_particle:
            extract_coords(pickable_object, run, "remoteobjects")


Run CopickRun(name=TS_5_4, len(voxel_spacings)=None, len(picks)=None, len(meshes)=None, len(segmentations)=None) at 0x154b9ef3a990
apo-ferritin


IndexError: list index out of range

In [14]:


# Make a copick project

config_blob = """{
    "name": "czii_cryoet_mlchallenge_2024",
    "description": "2024 CZII CryoET ML Challenge training data.",
    "version": "1.0.0",

    "pickable_objects": [
        {
            "name": "apo-ferritin",
            "is_particle": true,
            "pdb_id": "4V1W",
            "label": 1,
            "color": [  0, 117, 220, 128],
            "radius": 60,
            "map_threshold": 0.0418
        },
        {
            "name": "beta-amylase",
            "is_particle": true,
            "pdb_id": "1FA2",
            "label": 2,
            "color": [153,  63,   0, 128],
            "radius": 65,
            "map_threshold": 0.035
        },
        {
            "name": "beta-galactosidase",
            "is_particle": true,
            "pdb_id": "6X1Q",
            "label": 3,
            "color": [ 76,   0,  92, 128],
            "radius": 90,
            "map_threshold": 0.0578
        },
        {
            "name": "ribosome",
            "is_particle": true,
            "pdb_id": "6EK0",
            "label": 4,
            "color": [  0,  92,  49, 128],
            "radius": 150,
            "map_threshold": 0.0374
        },
        {
            "name": "thyroglobulin",
            "is_particle": true,
            "pdb_id": "6SCJ",
            "label": 5,
            "color": [ 43, 206,  72, 128],
            "radius": 130,
            "map_threshold": 0.0278
        },
        {
            "name": "virus-like-particle",
            "is_particle": true,
            "pdb_id": "6N4V",            
            "label": 6,
            "color": [255, 204, 153, 128],
            "radius": 135,
            "map_threshold": 0.201
        }
    ],

    "overlay_root": "test/overlay",

    "overlay_fs_args": {
        "auto_mkdir": true
    },

    "static_root": "/bigwork/nhwpajjy/competition/cryoET-data/czii-cryo-et-object-identification/test/static"
}"""

copick_config_path = "test_copick.config"


with open(copick_config_path, "w") as f:
    f.write(config_blob)
    



In [45]:
import csv
import os

os.listdir("/bigwork/nhwpajjy/competition/cryoET-data/czii-cryo-et-object-identification/test/static/ExperimentRuns")

copick_root = copick.from_file(config)


results = []
pick_id = 0

# id,experiment,particle_type,x,y,z
for run in copick_root.runs:
    run_id = run.meta.name
    for particle_type in copick_root.pickable_objects:
        picks = run.get_picks(particle_type.name, user_id="deepfindET")
        if picks:
            picks = picks[0]
            points = picks.points
            for point in points:                
                row = [pick_id, run_id, particle_type.name, point.location.x, point.location.y, point.location.z]
                results.append(row)
                pick_id += 1

print(f"Found {len(results)} picks")

# Define CSV output file path
output_csv_path = "submission.csv"

# Write results to CSV
with open(output_csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write header
    writer.writerow(["id", "experiment", "particle_type", "x", "y", "z"])
    # Write data rows
    writer.writerows(results)

Found 7809 picks


In [41]:
import copick
copickRoot = copick.from_file(copick_config_path)
tomo_ids = [run.name for run in copickRoot.runs]
targets = [obj.name for obj in copickRoot.pickable_objects if obj.is_particle]


In [52]:
ground_truth = []
for tomoID in tomo_ids:
    print(tomoID)
    copickRun = copickRoot.get_run(tomoID)
    for target_name in targets:
        picks = copickRun.get_picks(
                    object_name=target_name,
                    user_id=None,
                    session_id=None,
                )
        if picks:
            picks = picks[0]
            points = picks.points
            for point in points:                
                row = [pick_id, tomoID, particle_type.name, point.location.x, point.location.y, point.location.z]
                ground_truth.append(row)
                pick_id += 1
print(f"Found {len(ground_truth)} picks")

output_csv_path = "solution.csv"

# Write results to CSV
with open(output_csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write header
    writer.writerow(["id", "experiment", "particle_type", "x", "y", "z"])
    # Write data rows
    writer.writerows(ground_truth)

TS_5_4
TS_69_2
TS_6_4
Found 7809 picks


In [43]:
objl_coords

[{'label': 1,
  'x': 620.1075268817204,
  'y': 624.7849462365591,
  'z': 63.86021505376344,
  'phi': 0,
  'psi': 0,
  'the': 0},
 {'label': 1,
  'x': 621.9867674858223,
  'y': 622.6767485822306,
  'z': 149.19659735349717,
  'phi': 0,
  'psi': 0,
  'the': 0},
 {'label': 1,
  'x': 549.1940298507462,
  'y': 511.44776119402985,
  'z': 3.671641791044776,
  'phi': 0,
  'psi': 0,
  'the': 0},
 {'label': 1,
  'x': 548.0674157303371,
  'y': 527.9775280898876,
  'z': 4.977528089887641,
  'phi': 0,
  'psi': 0,
  'the': 0},
 {'label': 1,
  'x': 581.1285714285714,
  'y': 460.4142857142857,
  'z': 25.12857142857143,
  'phi': 0,
  'psi': 0,
  'the': 0},
 {'label': 1,
  'x': 576.9565217391304,
  'y': 464.4782608695652,
  'z': 31.858695652173914,
  'phi': 0,
  'psi': 0,
  'the': 0},
 {'label': 1,
  'x': 567.2112676056338,
  'y': 467.6056338028169,
  'z': 12.0,
  'phi': 0,
  'psi': 0,
  'the': 0},
 {'label': 1,
  'x': 552.6319290465632,
  'y': 482.61640798226165,
  'z': 8.350332594235033,
  'phi': 0,
  